<a href="https://colab.research.google.com/github/avocadopelvis/mental-load/blob/main/mental_load_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

In [45]:
# #LOAD DATA
# df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
# df = df.drop(df.columns[0], axis = 1)
# df

In [3]:
# use gpu if available 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [48]:
df = pd.read_csv("/content/drive/MyDrive/MENTAL-LOAD-DATASET/final.csv")
df = df.drop(df.columns[0], axis = 1)

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, random_state = 42)

In [50]:
class SignalDataset(Dataset):
    def __init__(self, df):
      # get x and convert to tensor
      self.x = torch.tensor(df.iloc[:, 0:3840].values)
      # get y and convert to tensor
      self.y = torch.tensor(df["label"].values)
      # get number of samples
      self.n_samples = df.shape[0]

    def __len__(self):
      return self.n_samples

    def __getitem__(self, index):
      return self.x[index], self.y[index]

In [51]:
train_dataset = SignalDataset(train)
test_dataset = SignalDataset(test)

In [52]:
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True) #,num_workers = 2
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [53]:
# dataiter = iter(test_dataloader)
# data = dataiter.next()
# features, labels = data
# # print(features, labels)
# features

In [54]:
# hyperparameters
input_size = 3840
hidden_size = 100
num_classes = 3
num_epochs = 15
batch_size = 10
learning_rate = 0.001

In [9]:
# from torch import nn

# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(28*28, 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, 10),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         logits = self.linear_relu_stack(x)
#         return logits

In [56]:
from torch import nn

class NeuralNetwork(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNetwork, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out

model = NeuralNetwork(input_size, hidden_size, num_classes)
print(model)

NeuralNetwork(
  (l1): Linear(in_features=3840, out_features=100, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=100, out_features=3, bias=True)
)


In [57]:
# LOSS & OPTIMIZER
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
# optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate, momentum=0.9)

In [58]:
# TRAINING LOOP
n_total_steps = len(train_dataloader)

for epoch in range(num_epochs):
  for i, data in enumerate(train_dataloader, 0):
    # get the inputs and labels
    inputs, labels = data

    # forward
    outputs = model(inputs.float())
    loss = criterion(outputs, labels)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%100 == 0:
      print(f'epoch {epoch+1} / {num_epochs}, step {i+1} / {n_total_steps}, loss = {loss.item():.4f}')

print("Finished Training")

epoch 1 / 15, step 100 / 147, loss = 1.2603
epoch 2 / 15, step 100 / 147, loss = 0.6349
epoch 3 / 15, step 100 / 147, loss = 0.3715
epoch 4 / 15, step 100 / 147, loss = 0.2836
epoch 5 / 15, step 100 / 147, loss = 0.1212
epoch 6 / 15, step 100 / 147, loss = 0.0241
epoch 7 / 15, step 100 / 147, loss = 0.0412
epoch 8 / 15, step 100 / 147, loss = 0.0067
epoch 9 / 15, step 100 / 147, loss = 0.0059
epoch 10 / 15, step 100 / 147, loss = 0.0123
epoch 11 / 15, step 100 / 147, loss = 0.0060
epoch 12 / 15, step 100 / 147, loss = 0.0039
epoch 13 / 15, step 100 / 147, loss = 0.0020
epoch 14 / 15, step 100 / 147, loss = 0.0013
epoch 15 / 15, step 100 / 147, loss = 0.0021
Finished Training


In [59]:
# TESTING
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for inputs, labels in test_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs.float())

    # value, index (We don't need value)
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()

  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')

accuracy = 38.095238095238095
